Cosine Similarity Of features based off of similarity of descriptions and Mood

In [18]:
import pandas as pd


df = pd.read_csv('Data/netfix_cleaned.csv')

In [19]:
df["mood"] = df["mood"].fillna("Unlabeled")

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize two vectorizers
vectorizer_desc = TfidfVectorizer(stop_words='english')
vectorizer_mood = TfidfVectorizer(stop_words='english')

# Fit and transform the data for each feature
tfidf_matrix_desc = vectorizer_desc.fit_transform(df['description'])
tfidf_matrix_mood = vectorizer_mood.fit_transform(df['mood'])

In [21]:
from scipy.sparse import hstack

# Combine the TF-IDF matrices
tfidf_matrix_combined = hstack([tfidf_matrix_desc, tfidf_matrix_mood])


In [22]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix_combined, tfidf_matrix_combined)


In [25]:
def recommend_movies(title, df, cosine_sim, top_n=5):
    # Find matches
    matches = df.index[df['description'].str.contains(title, case=False, na=False)].tolist()
    
    # Check if any matches were found
    if not matches:
        return f"No movies found matching the title: {title}"
    
    # Assuming there's at least one match, proceed as before
    idx = matches[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # Get top N matches
    movie_indices = [i[0] for i in sim_scores]
    return df['description'].iloc[movie_indices]

# Test the recommendation system with a valid and invalid title
print(recommend_movies("Ted", df, cosine_sim, top_n=2))
print(recommend_movies("asdfl;j", df, cosine_sim, top_n=2))



522    An exalted but short-fused surgeon plunges int...
20     In the companion to 2013's "Bombay Talkies," f...
Name: description, dtype: object
No movies found matching the title: asdfl;j
